In [21]:
from __future__ import print_function
import os.path
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import pandas as pd

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

# The ID and range of a sample spreadsheet.
SPREADSHEET_ID = '1SuwNOxpQqEAM8yoJZnCPywFZ_SlR23otGhbGqtwzONQ'
RANGE_NAME = 'revenue'


def main():
    """Shows basic usage of the Sheets API.
    Prints values from a sample spreadsheet.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('../key/rev_token.json'):
        creds = Credentials.from_authorized_user_file('../key/rev_token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                '../key/OAuth.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('../key/rev_token.json', 'w') as token:
            token.write(creds.to_json())

    try:
        service = build('sheets', 'v4', credentials=creds)

        # Call the Sheets API
        sheet = service.spreadsheets()
        result = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                                    range=RANGE_NAME).execute()
        values = result.get('values', [])

        df = pd.DataFrame(values)

        df_2 = df.melt(id_vars=[1], var_name="date", value_name="Value")

        print(df_2)
        #cleaning columns
        #replace = {' ' : '_', '-' : '_', '.' : '_', '?' : '', '[' : '', ']' : '', '(' : '', ')' : ''}

        #def clean_col (col, clean_col):
        #    col.columns = col.columns.str.lower()
        #    for i, j in clean_col.items():
        #        col.columns = col.columns.str.replace(i, j)
        #    return col
        
        #df_2 = clean_col(df, replace)

        #df_2


    except HttpError as err:
        print(err)


if __name__ == '__main__':
    main()

                                              1  date        Value
0                                   Nama Proyek     0           No
1               Benioff Ocean Initiatives (BOI)     0            1
2                 Advisory TPS 3R Wilayah Timur     0            2
3             River Waste Study (Solo-Surabaya)     0            3
4                          Waste Picker Wage PP     0            4
...                                         ...   ...          ...
6583              Akabis Class ABeam Consulting    36         None
6584                Akabis Xperience GIZ CAPSEA    36         None
6585  Akabis Workshop PT Pasar Jasa Profesional    36         None
6586                         Akabis Class Unido    36         None
6587                                               36  952.640.750

[6588 rows x 3 columns]
